In [1]:
import datetime
import pandas as pd
import gymnasium as gym
import gym_anytrading
import numpy as np
import datetime
from gym_anytrading.envs import StocksEnv
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout 
from keras.optimizers import Adam
import subprocess

In [2]:
import pandas as pd
from finta import TA
from more_environment import CustomTradingEnv

df = gym_anytrading.datasets.STOCKS_GOOGL.copy()
df.shape

df.info()

df['TEMA'] = TA.TEMA(df, 30) # Triple Exponential Moving Average --> in ce directie se indreapta pretul pe parcursul a 12 zile cu accent mare pe ultimele zile
df['ER'] = TA.ER(df) # Kaufman Efficiency Indicator --> confirmare a trendului (-1,1) : -1 downtrend, 1 uptrend, aprox 0 random (trend confirmation)
df['RSI'] = TA.RSI(df) # Relative Strength Index --> intre 0 si 100, 30-70 intervalul de interes, 30-0 oversold, 70-100 overbought (trend confirmation)
df['OBV'] = TA.OBV(df) # On Balance Volume --> confirmare a trendului (trend confirmation) --> se bazeaza pe volumul tranzactiilor
df['STOCH'] = TA.STOCH(df) # Stochastic Oscillator --> confirmare a trendului (trend confirmation) --> intre 0 si 100, 20-80 intervalul de interes, 20-0 oversold, 80-100 overbought
df.fillna(0, inplace=True)
df.head(20)

# Define window_size based on the number of historical data points you want to consider
window_size = 30

# Split the dataframe into training and testing sets
train_df = df.iloc[:int(0.8 * len(df))]
test_df = df.iloc[int(0.8 * len(df)):]

# Define your frame_bound based on the training data
frame_bound_train = (window_size, len(train_df) - 1)

# Create your custom trading environment for training
train_env = CustomTradingEnv(train_df, window_size, frame_bound_train, render_mode=None)

# Define your frame_bound based on the testing data
frame_bound_test = (window_size, len(test_df) - 1)

# Create your custom trading environment for testing
test_env = CustomTradingEnv(test_df, window_size, frame_bound_test, render_mode=None)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2335 entries, 2009-05-22 to 2018-08-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2335 non-null   float64
 1   High       2335 non-null   float64
 2   Low        2335 non-null   float64
 3   Close      2335 non-null   float64
 4   Adj Close  2335 non-null   float64
 5   Volume     2335 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 127.7 KB


In [6]:
%load_ext autoreload
%autoreload 2

# test reward function
from tqdm import tqdm
import numpy as np
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from dqn_2 import Model


def evaluate_agent(test_env, model, episodes):
    # Create your custom trading environment for testing
    mean_reward, mean_profit = 0, 0
    profits = []
    rewards = []
    for episode in range(episodes):
        current_state = test_env.reset()[0]
        while True:
            action, q_values = model.predict(torch.tensor(current_state, dtype=torch.float32).unsqueeze(0))

            next_state, reward, deprecated, done, info = test_env.step(action)
            if done:
                print(info)
                break
            current_state = next_state
            
        mean_profit += info['total_profit']
        mean_reward += info['total_reward']
        profits.append(info['total_profit'])
        rewards.append(info['total_reward'])
        
    mean_reward /= episodes
    mean_profit /= episodes
    
    return mean_reward, mean_profit, mean_profit/mean_reward, profits, rewards


test_env = CustomTradingEnv(test_df, window_size, frame_bound_test, render_mode=None)

possible_actions = [0, 1]

# Assuming your model is already trained and saved as weight_episode_300.pt
trained_model = Model(input_size=10, hidden_size=64, num_layers=2, output_size=len(possible_actions), dropout=0.2)
trained_model.load_state_dict(torch.load('weights_episode_400.pt'))
trained_model.eval()

# Create the test environment
test_env = CustomTradingEnv(test_df, window_size, frame_bound_test, render_mode=None)

# Call the evaluate_agent function with the trained model
mean_reward, mean_profit, fr, profits, rewards = evaluate_agent(test_env, trained_model, episodes=100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 'position': <Positions.Long: 1>}
{'total_reward': 452.89001400000006, 'total_profit': 1.532749961633769, 

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from dqn_2 import Agent, Utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define Training Parameters
num_episodes = 1000
initial_epsilon = 0.99
starting_mem_len = 50000  # Adjust as needed
max_mem_len = 1000000  # Adjust as needed
learn_rate = 0.001

# Initialize DQN Agent
possible_actions = [0, 1]  # Adjust based on your action space
agent = Agent(possible_actions, starting_mem_len, max_mem_len, initial_epsilon, learn_rate, device)

# Training Loop
utils = Utils()

for episode in range(1, num_episodes + 1):
    episode_score, info = utils.play_episode(train_env, agent, episode, debug=False)
    print(f"Episode {episode}/{num_episodes} - Score: {episode_score}, Profit: {info['total_profit']}")

    # Save Model Weights periodically
    if episode % 100 == 0:
        torch.save(agent.model.state_dict(), f'weights_episode_{episode}.pt')
        print(f'\nWeights saved for episode {episode}')

    
# Save Model Weights
torch.save(agent.model.state_dict(), 'final_weights.pt')
print('\nFinal weights saved!')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cuda

Agent Initialized

Episode 1/1000 - Score: -178.8205730000003, Profit: 0.0021237098818859255
Episode 2/1000 - Score: -133.30923900000002, Profit: 0.001238747859726668
Episode 3/1000 - Score: 278.0689159999993, Profit: 0.0023079231718086653
Episode 4/1000 - Score: -292.45435399999894, Profit: 0.0016588959459746648
Episode 5/1000 - Score: 314.1152050000006, Profit: 0.0028126741812930242
Episode 6/1000 - Score: 157.4782570000006, Profit: 0.0022173867950512805
Episode 7/1000 - Score: -4.080163999999883, Profit: 0.0018680347507787156
Episode 8/1000 - Score: 333.60865800000045, Profit: 0.002931309520276743
Episode 9/1000 - Score: -11.402795999999512, Profit: 0.002432069071616467
Episode 10/1000 - Score: -105.89847899999955, Profit: 0.00177722456327277
Episode 11/1000 - Score: -158.03055899999993, Profit: 0.0019381319752780285
Episode 12/1000 - Score: 42.70738699999873, Profit: 0.002064

c:\Users\alber\Desktop\trading-with-reinforcement-learning\google_train\dqn_2.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(labels, dtype=torch.float32).to(self.device))
c:\Users\alber\Desktop\trading-with-reinforcement-learning\google_train\dqn_2.py:235: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(labels, dtype=torch.float32).to(self.device))



Weights saved!
Episode 28/1000 - Score: -47.580877999999444, Profit: 0.0016734845279237072
Episode 29/1000 - Score: -29.368766999999934, Profit: 0.002307960285920684
Episode 30/1000 - Score: 141.26069400000063, Profit: 0.0025497544378655
Episode 31/1000 - Score: 394.6779239999988, Profit: 0.0018372415571661538
Episode 32/1000 - Score: -341.56979400000057, Profit: 0.001405137436268166

Target model updated
Episode 33/1000 - Score: -220.17779100000052, Profit: 0.002277371141377252
Episode 34/1000 - Score: 113.91818300000003, Profit: 0.002331748014489435
Episode 35/1000 - Score: -396.1913199999982, Profit: 0.001321146896240128
Episode 36/1000 - Score: 121.54941500000115, Profit: 0.0024879933905910703
Episode 37/1000 - Score: 82.44617400000047, Profit: 0.0023568089778077646
Episode 38/1000 - Score: 76.5679169999998, Profit: 0.0035623514467042327

Target model updated
Episode 39/1000 - Score: -84.30173700000137, Profit: 0.004265950341811433
Episode 40/1000 - Score: -1.5057430000006775, Pro

KeyboardInterrupt: 